##  Banking Binary Classification Problem

### Problem
We would like to predict if given a bank client, are they likely to make a term deposit or not at the bank.

Dataset: bankingInfo.csv
     - if output == yes, then the client has made a term deposit at this bank, else they have not.

Project was built using the bc_ml_breast_cancer_pred.ipynb and the deep_learning.ipynb files provided by Dr. Sambriddhi Mainali along with the *A hands-on introduction to feed-forward neural networks using Tensorflow and Keras* Github Repository by Dr. Badri Adhikari found [here.](https://badriadhikari.github.io/AI-2022spring/NN-using-TF.html) 

### Phase 2: Build a Model to Overfit the Entire Dataset
____

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import keras
import tensorflow 
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

In [24]:
#low_memory=False used to remove the warning for mixed data types in each columns
banking_data = pd.read_csv('./cleaned_banking_data.csv', low_memory=False)
banking_data.head()

,age,education,housing,loan,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,...,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,output
0,36,2,0,0,9,999,0,0,1.4,93.444,...,0,1,0,0,0,0,0,0,0,0
1,43,1,0,0,2,999,0,0,1.1,93.994,...,0,0,0,0,0,0,0,0,0,0
2,27,4,1,0,3,999,0,0,1.4,93.918,...,0,0,0,0,0,1,0,0,0,0
3,28,3,1,0,3,999,0,0,1.1,93.994,...,0,0,0,0,0,0,0,0,0,0
4,25,4,1,0,5,999,0,0,1.4,93.918,...,0,0,0,0,0,0,0,0,0,0


In [25]:
original_dataset.shape

(8516, 28)

#### 2.1 Normalize the Data

In [28]:
# Shuffle dataframe in-place and reset the index
banking_data = banking_data.sample(frac=1)
banking_data.head()

,age,education,housing,loan,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,...,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,output
472,59,4,0,0,2,999,0,0,1.4,93.918,...,0,0,0,1,0,0,0,0,0,0
7489,44,4,1,0,2,6,1,1,-3.4,92.649,...,0,0,0,0,0,0,0,0,0,1
7481,48,6,1,0,1,3,1,1,-3.4,92.649,...,0,0,1,0,0,0,0,0,0,1
4677,37,4,1,0,1,999,0,0,1.4,93.918,...,0,0,1,0,0,0,0,0,0,1
7017,74,6,1,1,3,999,0,0,-2.9,92.201,...,0,0,0,1,0,0,0,0,0,1


#### 2.2 Build a Model with 1 Neuron

In [43]:
X = banking_data.drop(['output'], axis = 'columns')
y = banking_data['output']

# Normalization of the data 
min = X.min(axis = 0) 
max = X.max(axis = 0) 
X = (X - min) / (max - min)

3570    0
1245    0
4458    1
4600    1
894     0
       ..
2310    0
3969    0
5215    1
7200    1
3333    0
Name: output, Length: 8516, dtype: int64


In [48]:
myadam = keras.optimizers.Adam(learning_rate=0.001)

# constructing a single neuron model
single_neuron = Sequential()
single_neuron.add(Dense(1, input_dim = len(X.columns), activation='sigmoid'))
print(single_neuron.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 1)                 27        
                                                                 
Total params: 27 (108.00 Byte)
Trainable params: 27 (108.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [50]:
#Check for cycles in the Sequential Model
single_neuron.compile(loss='binary_crossentropy', optimizer=myadam, metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'single_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_single = single_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
842/852 [============================>.] - ETA: 0s - loss: 0.7636 - accuracy: 0.4660
Epoch 1: val_loss improved from inf to 0.66898, saving model to single_model.hdf5
852/852 [==============================] - 3s 3ms/step - loss: 0.7624 - accuracy: 0.4677 - val_loss: 0.6690 - val_accuracy: 0.5969
Epoch 2/150
841/852 [============================>.] - ETA: 0s - loss: 0.6339 - accuracy: 0.6620
Epoch 2: val_loss improved from 0.66898 to 0.60968, saving model to single_model.hdf5
852/852 [==============================] - 2s 3ms/step - loss: 0.6334 - accuracy: 0.6628 - val_loss: 0.6097 - val_accuracy: 0.6896
Epoch 3/150
831/852 [============================>.] - ETA: 0s - loss: 0.5959 - accuracy: 0.7072
Epoch 3: val_loss improved from 0.60968 to 0.58593, saving model to single_model.hdf5
852/852 [==============================] - 2s 3ms/step - loss: 0.5955 - accuracy: 0.7069 - val_loss: 0.5859 - val_accuracy: 0.7112
Epoch 4/150
840/852 [============================>.] - ETA: 0s

In [51]:
#Check for cycles in the Sequential Model
history_single = single_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
832/852 [============================>.] - ETA: 0s - loss: 0.5614 - accuracy: 0.7215
Epoch 1: val_loss improved from 0.56015 to 0.56014, saving model to single_model.hdf5
852/852 [==============================] - 2s 3ms/step - loss: 0.5609 - accuracy: 0.7226 - val_loss: 0.5601 - val_accuracy: 0.7236
Epoch 2/150
838/852 [============================>.] - ETA: 0s - loss: 0.5610 - accuracy: 0.7212
Epoch 2: val_loss did not improve from 0.56014
852/852 [==============================] - 2s 3ms/step - loss: 0.5606 - accuracy: 0.7219 - val_loss: 0.5608 - val_accuracy: 0.7246
Epoch 3/150
845/852 [============================>.] - ETA: 0s - loss: 0.5611 - accuracy: 0.7228
Epoch 3: val_loss did not improve from 0.56014
852/852 [==============================] - 2s 3ms/step - loss: 0.5609 - accuracy: 0.7232 - val_loss: 0.5602 - val_accuracy: 0.7244
Epoch 4/150
839/852 [============================>.] - ETA: 0s - loss: 0.5609 - accuracy: 0.7232
Epoch 4: val_loss did not improve from 

In [52]:
single_neuron.load_weights('single_model.hdf5')

In [53]:
# Other Metrics
single_neuron_preds = single_neuron.predict(X)
accuracy = accuracy_score(y,single_neuron_preds.round())
precision = precision_score(y, single_neuron_preds.round())
recall = recall_score(y, single_neuron_preds.round())
f1score = f1_score(y, single_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 1ms/step
Accuracy: 0.73
Precision: 0.75
Recall: 0.67
F1-score: 0.71


#### 2.3 Build a Model with 3 Neurons in 2 Layers

In [54]:
# constructing a 3 neuron model
three_neuron = Sequential()
three_neuron.add(Dense(2, input_dim = len(X.columns), activation='relu'))
three_neuron.add(Dense(1, activation='sigmoid'))
print(three_neuron.summary())

#Check for cycles in the Sequential Model
three_neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'three_neuron_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_3 = three_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 2)                 54        
                                                                 
 dense_11 (Dense)            (None, 1)                 3         
                                                                 
Total params: 57 (228.00 Byte)
Trainable params: 57 (228.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
837/852 [============================>.] - ETA: 0s - loss: 0.6471 - accuracy: 0.6296
Epoch 1: val_loss improved from inf to 0.61003, saving model to three_neuron_model.hdf5
852/852 [==============================] - 3s 3ms/step - loss: 0.6471 - accuracy: 0.6301 - val_loss: 0.6100 - val_accuracy: 0.7053
Epoch 2/150
847/852 [============================>.] - ETA: 0s - loss: 0.5934 - accuracy: 0.

In [65]:
# constructing a 3 neuron model
history_3 = three_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
831/852 [============================>.] - ETA: 0s - loss: 0.5490 - accuracy: 0.7327
Epoch 1: val_loss did not improve from 0.42561
852/852 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7317 - val_loss: 0.5508 - val_accuracy: 0.7318
Epoch 2/150
835/852 [============================>.] - ETA: 0s - loss: 0.5511 - accuracy: 0.7329
Epoch 2: val_loss did not improve from 0.42561
852/852 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7334 - val_loss: 0.5488 - val_accuracy: 0.7351
Epoch 3/150
834/852 [============================>.] - ETA: 0s - loss: 0.5486 - accuracy: 0.7350
Epoch 3: val_loss did not improve from 0.42561
852/852 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7336 - val_loss: 0.5490 - val_accuracy: 0.7346
Epoch 4/150
839/852 [============================>.] - ETA: 0s - loss: 0.5509 - accuracy: 0.7316
Epoch 4: val_loss did not improve from 0.42561
852/852 [======================

In [66]:
three_neuron.load_weights('three_neuron_model.hdf5')

In [67]:
# Other Metrics
three_neuron_preds = three_neuron.predict(X)
accuracy = accuracy_score(y,three_neuron_preds.round())
precision = precision_score(y, three_neuron_preds.round())
recall = recall_score(y, three_neuron_preds.round())
f1score = f1_score(y, three_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 867us/step
Accuracy: 0.73
Precision: 0.79
Recall: 0.64
F1-score: 0.71


#### 2.4 Build a Model with 7 Neurons in 3 Layers

In [55]:
# constructing a 7 neuron model
seven_neuron = Sequential()
seven_neuron.add(Dense(4, input_dim = len(X.columns), activation='relu'))
seven_neuron.add(Dense(2, activation='relu'))
seven_neuron.add(Dense(1, activation='sigmoid'))
print(seven_neuron.summary())

#Check for cycles in the Sequential Model
seven_neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'seven_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_7 = seven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 4)                 108       
                                                                 
 dense_13 (Dense)            (None, 2)                 10        
                                                                 
 dense_14 (Dense)            (None, 1)                 3         
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
833/852 [============================>.] - ETA: 0s - loss: 0.6499 - accuracy: 0.6627
Epoch 1: val_loss improved from inf to 0.61962, saving model to seven_model.hdf5
852/852 [==============================] - 3s 3ms/step - loss: 0.6493 - accuracy: 0.6640 - val_l

In [56]:
history_7 = seven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
832/852 [============================>.] - ETA: 0s - loss: 0.5394 - accuracy: 0.7367
Epoch 1: val_loss did not improve from 0.53653
852/852 [==============================] - 3s 3ms/step - loss: 0.5387 - accuracy: 0.7379 - val_loss: 0.5408 - val_accuracy: 0.7379
Epoch 2/150
835/852 [============================>.] - ETA: 0s - loss: 0.5392 - accuracy: 0.7393
Epoch 2: val_loss did not improve from 0.53653
852/852 [==============================] - 3s 3ms/step - loss: 0.5395 - accuracy: 0.7393 - val_loss: 0.5375 - val_accuracy: 0.7406
Epoch 3/150
847/852 [============================>.] - ETA: 0s - loss: 0.5394 - accuracy: 0.7387
Epoch 3: val_loss did not improve from 0.53653
852/852 [==============================] - 2s 3ms/step - loss: 0.5394 - accuracy: 0.7388 - val_loss: 0.5373 - val_accuracy: 0.7397
Epoch 4/150
827/852 [============================>.] - ETA: 0s - loss: 0.5391 - accuracy: 0.7416
Epoch 4: val_loss did not improve from 0.53653
852/852 [======================

In [59]:
history_7 = seven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
826/852 [============================>.] - ETA: 0s - loss: 0.5383 - accuracy: 0.7407
Epoch 1: val_loss did not improve from 0.51923
852/852 [==============================] - 2s 3ms/step - loss: 0.5390 - accuracy: 0.7400 - val_loss: 0.5377 - val_accuracy: 0.7393
Epoch 2/150
838/852 [============================>.] - ETA: 0s - loss: 0.5392 - accuracy: 0.7397
Epoch 2: val_loss did not improve from 0.51923
852/852 [==============================] - 3s 3ms/step - loss: 0.5392 - accuracy: 0.7400 - val_loss: 0.5363 - val_accuracy: 0.7420
Epoch 3/150
846/852 [============================>.] - ETA: 0s - loss: 0.5393 - accuracy: 0.7416
Epoch 3: val_loss did not improve from 0.51923
852/852 [==============================] - 2s 3ms/step - loss: 0.5394 - accuracy: 0.7415 - val_loss: 0.5362 - val_accuracy: 0.7408
Epoch 4/150
852/852 [==============================] - ETA: 0s - loss: 0.5389 - accuracy: 0.7398
Epoch 4: val_loss did not improve from 0.51923
852/852 [======================

In [83]:
seven_neuron.load_weights('seven_model.hdf5')

In [84]:
# Other Metrics
seven_neuron_preds = seven_neuron.predict(X)
accuracy = accuracy_score(y,seven_neuron_preds.round())
precision = precision_score(y, seven_neuron_preds.round())
recall = recall_score(y, seven_neuron_preds.round())
f1score = f1_score(y, seven_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 946us/step
Accuracy: 0.74
Precision: 0.83
Recall: 0.61
F1-score: 0.70


#### 2.5 Build a Model with 15 Neurons in 4 Layers

In [57]:
# constructing a 15 neuron model
fifteen_neuron = Sequential()
fifteen_neuron.add(Dense(8, input_dim = len(X.columns), activation='relu'))
fifteen_neuron.add(Dense(4, activation='relu'))
fifteen_neuron.add(Dense(2, activation='relu'))
fifteen_neuron.add(Dense(1, activation='sigmoid'))
print(fifteen_neuron.summary())

#Check for cycles in the Sequential Model
fifteen_neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'fifteen_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_15 = fifteen_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 8)                 216       
                                                                 
 dense_16 (Dense)            (None, 4)                 36        
                                                                 
 dense_17 (Dense)            (None, 2)                 10        
                                                                 
 dense_18 (Dense)            (None, 1)                 3         
                                                                 
Total params: 265 (1.04 KB)
Trainable params: 265 (1.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
828/852 [============================>.] - ETA: 0s - loss: 0.6428 - accuracy: 0.6099
Epoch 1: val_loss improved from inf to 0.57055, savin

In [58]:
history_15 = fifteen_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
838/852 [============================>.] - ETA: 0s - loss: 0.5282 - accuracy: 0.7488
Epoch 1: val_loss did not improve from 0.52408
852/852 [==============================] - 3s 3ms/step - loss: 0.5274 - accuracy: 0.7494 - val_loss: 0.5271 - val_accuracy: 0.7501
Epoch 2/150
825/852 [============================>.] - ETA: 0s - loss: 0.5286 - accuracy: 0.7473
Epoch 2: val_loss did not improve from 0.52408
852/852 [==============================] - 2s 3ms/step - loss: 0.5284 - accuracy: 0.7474 - val_loss: 0.5271 - val_accuracy: 0.7472
Epoch 3/150
841/852 [============================>.] - ETA: 0s - loss: 0.5296 - accuracy: 0.7458
Epoch 3: val_loss improved from 0.52408 to 0.52336, saving model to fifteen_model.hdf5
852/852 [==============================] - 3s 3ms/step - loss: 0.5285 - accuracy: 0.7469 - val_loss: 0.5234 - val_accuracy: 0.7508
Epoch 4/150
825/852 [============================>.] - ETA: 0s - loss: 0.5273 - accuracy: 0.7497
Epoch 4: val_loss did not improve from

In [60]:
history_15 = fifteen_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
833/852 [============================>.] - ETA: 0s - loss: 0.5220 - accuracy: 0.7508
Epoch 1: val_loss did not improve from 0.51923
852/852 [==============================] - 3s 3ms/step - loss: 0.5232 - accuracy: 0.7502 - val_loss: 0.5217 - val_accuracy: 0.7515
Epoch 2/150
836/852 [============================>.] - ETA: 0s - loss: 0.5235 - accuracy: 0.7504
Epoch 2: val_loss did not improve from 0.51923
852/852 [==============================] - 3s 3ms/step - loss: 0.5237 - accuracy: 0.7500 - val_loss: 0.5211 - val_accuracy: 0.7508
Epoch 3/150
840/852 [============================>.] - ETA: 0s - loss: 0.5234 - accuracy: 0.7479
Epoch 3: val_loss did not improve from 0.51923
852/852 [==============================] - 2s 3ms/step - loss: 0.5235 - accuracy: 0.7480 - val_loss: 0.5206 - val_accuracy: 0.7521
Epoch 4/150
852/852 [==============================] - ETA: 0s - loss: 0.5233 - accuracy: 0.7493
Epoch 4: val_loss did not improve from 0.51923
852/852 [======================

In [68]:
fifteen_neuron.load_weights('fifteen_model.hdf5')

In [69]:
# Other Metrics
fifteen_neuron_preds = fifteen_neuron.predict(X)
accuracy = accuracy_score(y,fifteen_neuron_preds.round())
precision = precision_score(y, fifteen_neuron_preds.round())
recall = recall_score(y, fifteen_neuron_preds.round())
f1score = f1_score(y, fifteen_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 939us/step
Accuracy: 0.75
Precision: 0.84
Recall: 0.62
F1-score: 0.72


#### 2.6 Build a Model with 31 Neurons in 5 Layers

In [61]:
# constructing a 31 neuron model
thirty_one_neuron = Sequential()
thirty_one_neuron.add(Dense(16, input_dim = len(X.columns), activation='relu'))
thirty_one_neuron.add(Dense(8, activation='relu'))
thirty_one_neuron.add(Dense(4, activation='relu'))
thirty_one_neuron.add(Dense(2, activation='relu'))
thirty_one_neuron.add(Dense(1, activation='sigmoid'))
print(thirty_one_neuron.summary())

#Check for cycles in the Sequential Model
thirty_one_neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'thirty_one_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_31 = thirty_one_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 16)                432       
                                                                 
 dense_20 (Dense)            (None, 8)                 136       
                                                                 
 dense_21 (Dense)            (None, 4)                 36        
                                                                 
 dense_22 (Dense)            (None, 2)                 10        
                                                                 
 dense_23 (Dense)            (None, 1)                 3         
                                                                 
Total params: 617 (2.41 KB)
Trainable params: 617 (2.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
824/85

In [62]:
history_31 = thirty_one_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
843/852 [============================>.] - ETA: 0s - loss: 0.5135 - accuracy: 0.7522
Epoch 1: val_loss did not improve from 0.50503
852/852 [==============================] - 3s 3ms/step - loss: 0.5133 - accuracy: 0.7520 - val_loss: 0.5069 - val_accuracy: 0.7609
Epoch 2/150
834/852 [============================>.] - ETA: 0s - loss: 0.5125 - accuracy: 0.7560
Epoch 2: val_loss improved from 0.50503 to 0.50360, saving model to thirty_one_model.hdf5
852/852 [==============================] - 3s 3ms/step - loss: 0.5129 - accuracy: 0.7559 - val_loss: 0.5036 - val_accuracy: 0.7612
Epoch 3/150
838/852 [============================>.] - ETA: 0s - loss: 0.5121 - accuracy: 0.7547
Epoch 3: val_loss did not improve from 0.50360
852/852 [==============================] - 3s 3ms/step - loss: 0.5124 - accuracy: 0.7546 - val_loss: 0.5097 - val_accuracy: 0.7547
Epoch 4/150
826/852 [============================>.] - ETA: 0s - loss: 0.5120 - accuracy: 0.7542
Epoch 4: val_loss did not improve f

In [70]:
thirty_one_neuron.load_weights('thirty_one_model.hdf5')

In [71]:
# Other Metrics
thirty_one_neuron_preds = thirty_one_neuron.predict(X)
accuracy = accuracy_score(y,thirty_one_neuron_preds.round())
precision = precision_score(y, thirty_one_neuron_preds.round())
recall = recall_score(y, thirty_one_neuron_preds.round())
f1score = f1_score(y, thirty_one_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 962us/step
Accuracy: 0.77
Precision: 0.84
Recall: 0.65
F1-score: 0.74


#### 2.6 Build a Model with 63 Neurons in 6 Layers

In [63]:
# constructing a 63 neuron model
sixty_three_neuron = Sequential()
sixty_three_neuron.add(Dense(32, input_dim = len(X.columns), activation='relu'))
sixty_three_neuron.add(Dense(16, activation='relu'))
sixty_three_neuron.add(Dense(8, activation='relu'))
sixty_three_neuron.add(Dense(4, activation='relu'))
sixty_three_neuron.add(Dense(2, activation='relu'))
sixty_three_neuron.add(Dense(1, activation='sigmoid'))
print(sixty_three_neuron.summary())

#Check for cycles in the Sequential Model
sixty_three_neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'sixty_three_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_63 = sixty_three_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 32)                864       
                                                                 
 dense_25 (Dense)            (None, 16)                528       
                                                                 
 dense_26 (Dense)            (None, 8)                 136       
                                                                 
 dense_27 (Dense)            (None, 4)                 36        
                                                                 
 dense_28 (Dense)            (None, 2)                 10        
                                                                 
 dense_29 (Dense)            (None, 1)                 3         
                                                                 
Total params: 1577 (6.16 KB)
Trainable params: 1577 (

In [64]:
history_63 = sixty_three_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
830/852 [============================>.] - ETA: 0s - loss: 0.4761 - accuracy: 0.7631
Epoch 1: val_loss improved from 0.46122 to 0.46049, saving model to sixty_three_model.hdf5
852/852 [==============================] - 2s 2ms/step - loss: 0.4754 - accuracy: 0.7640 - val_loss: 0.4605 - val_accuracy: 0.7710
Epoch 2/150
830/852 [============================>.] - ETA: 0s - loss: 0.4726 - accuracy: 0.7673
Epoch 2: val_loss did not improve from 0.46049
852/852 [==============================] - 2s 2ms/step - loss: 0.4731 - accuracy: 0.7667 - val_loss: 0.4827 - val_accuracy: 0.7640
Epoch 3/150
850/852 [============================>.] - ETA: 0s - loss: 0.4716 - accuracy: 0.7661
Epoch 3: val_loss did not improve from 0.46049
852/852 [==============================] - 2s 2ms/step - loss: 0.4718 - accuracy: 0.7659 - val_loss: 0.4642 - val_accuracy: 0.7709
Epoch 4/150
829/852 [============================>.] - ETA: 0s - loss: 0.4712 - accuracy: 0.7680
Epoch 4: val_loss did not improve 

In [85]:
sixty_three_neuron.load_weights('sixty_three_model.hdf5')

In [86]:
# Other Metrics
sixty_three_neuron_preds = sixty_three_neuron.predict(X)
accuracy = accuracy_score(y,sixty_three_neuron_preds.round())
precision = precision_score(y, sixty_three_neuron_preds.round())
recall = recall_score(y, sixty_three_neuron_preds.round())
f1score = f1_score(y, sixty_three_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 991us/step
Accuracy: 0.79
Precision: 0.84
Recall: 0.71
F1-score: 0.77


#### 2.7 Build a Model with 125 Neurons in 6 Layers

In [73]:
# constructing a 125 neuron model
one_twentyfive_neuron = Sequential()
one_twentyfive_neuron.add(Dense(64, input_dim = len(X.columns), activation='relu'))
one_twentyfive_neuron.add(Dense(32, activation='relu'))
one_twentyfive_neuron.add(Dense(16, activation='relu'))
one_twentyfive_neuron.add(Dense(8, activation='relu'))
one_twentyfive_neuron.add(Dense(4, activation='relu'))
one_twentyfive_neuron.add(Dense(1, activation='sigmoid'))
print(one_twentyfive_neuron.summary())

#Check for cycles in the Sequential Model
one_twentyfive_neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'one_twentyfive_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_125 = one_twentyfive_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 64)                1728      
                                                                 
 dense_37 (Dense)            (None, 32)                2080      
                                                                 
 dense_38 (Dense)            (None, 16)                528       
                                                                 
 dense_39 (Dense)            (None, 8)                 136       
                                                                 
 dense_40 (Dense)            (None, 4)                 36        
                                                                 
 dense_41 (Dense)            (None, 1)                 5         
                                                                 
Total params: 4513 (17.63 KB)
Trainable params: 4513 

In [74]:
history_125 = one_twentyfive_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
817/852 [===========================>..] - ETA: 0s - loss: 0.4005 - accuracy: 0.8086
Epoch 1: val_loss did not improve from 0.38444
852/852 [==============================] - 2s 2ms/step - loss: 0.4032 - accuracy: 0.8066 - val_loss: 0.4075 - val_accuracy: 0.8084
Epoch 2/150
829/852 [============================>.] - ETA: 0s - loss: 0.4045 - accuracy: 0.8099
Epoch 2: val_loss did not improve from 0.38444
852/852 [==============================] - 2s 2ms/step - loss: 0.4034 - accuracy: 0.8101 - val_loss: 0.3890 - val_accuracy: 0.8158
Epoch 3/150
844/852 [============================>.] - ETA: 0s - loss: 0.4004 - accuracy: 0.8116
Epoch 3: val_loss did not improve from 0.38444
852/852 [==============================] - 2s 2ms/step - loss: 0.3994 - accuracy: 0.8121 - val_loss: 0.3869 - val_accuracy: 0.8173
Epoch 4/150
847/852 [============================>.] - ETA: 0s - loss: 0.3996 - accuracy: 0.8128
Epoch 4: val_loss did not improve from 0.38444
852/852 [======================

In [75]:
history_125 = one_twentyfive_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
844/852 [============================>.] - ETA: 0s - loss: 0.3373 - accuracy: 0.8438
Epoch 1: val_loss did not improve from 0.31743
852/852 [==============================] - 2s 2ms/step - loss: 0.3374 - accuracy: 0.8441 - val_loss: 0.3227 - val_accuracy: 0.8505
Epoch 2/150
823/852 [===========================>..] - ETA: 0s - loss: 0.3480 - accuracy: 0.8360
Epoch 2: val_loss did not improve from 0.31743
852/852 [==============================] - 2s 2ms/step - loss: 0.3458 - accuracy: 0.8371 - val_loss: 0.3207 - val_accuracy: 0.8506
Epoch 3/150
843/852 [============================>.] - ETA: 0s - loss: 0.3369 - accuracy: 0.8397
Epoch 3: val_loss did not improve from 0.31743
852/852 [==============================] - 2s 2ms/step - loss: 0.3360 - accuracy: 0.8403 - val_loss: 0.3235 - val_accuracy: 0.8463
Epoch 4/150
818/852 [===========================>..] - ETA: 0s - loss: 0.3372 - accuracy: 0.8439
Epoch 4: val_loss improved from 0.31743 to 0.31557, saving model to one_twenty

In [76]:
history_125 = one_twentyfive_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
836/852 [============================>.] - ETA: 0s - loss: 0.3205 - accuracy: 0.8502
Epoch 1: val_loss did not improve from 0.28657
852/852 [==============================] - 2s 2ms/step - loss: 0.3203 - accuracy: 0.8505 - val_loss: 0.2881 - val_accuracy: 0.8685
Epoch 2/150
835/852 [============================>.] - ETA: 0s - loss: 0.3026 - accuracy: 0.8584
Epoch 2: val_loss did not improve from 0.28657
852/852 [==============================] - 2s 2ms/step - loss: 0.3051 - accuracy: 0.8573 - val_loss: 0.2946 - val_accuracy: 0.8664
Epoch 3/150
840/852 [============================>.] - ETA: 0s - loss: 0.3105 - accuracy: 0.8600
Epoch 3: val_loss improved from 0.28657 to 0.28574, saving model to one_twentyfive_model.hdf5
852/852 [==============================] - 2s 2ms/step - loss: 0.3094 - accuracy: 0.8605 - val_loss: 0.2857 - val_accuracy: 0.8727
Epoch 4/150
822/852 [===========================>..] - ETA: 0s - loss: 0.3092 - accuracy: 0.8554
Epoch 4: val_loss did not impro

In [87]:
one_twentyfive_neuron.load_weights('one_twentyfive_model.hdf5')

In [88]:
# Other Metrics
one_twentyfive_neuron_preds = one_twentyfive_neuron.predict(X)
accuracy = accuracy_score(y, one_twentyfive_neuron_preds.round())
precision = precision_score(y, one_twentyfive_neuron_preds.round())
recall = recall_score(y, one_twentyfive_neuron_preds.round())
f1score = f1_score(y, one_twentyfive_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 1ms/step
Accuracy: 0.88
Precision: 0.94
Recall: 0.80
F1-score: 0.87


#### 2.8 Build a Model with 249 Neurons in 6 Layers

In [77]:
# constructing a 249 neuron model
two_fortynine_neuron = Sequential()
two_fortynine_neuron.add(Dense(128, input_dim = len(X.columns), activation='relu'))
two_fortynine_neuron.add(Dense(64, activation='relu'))
two_fortynine_neuron.add(Dense(32, activation='relu'))
two_fortynine_neuron.add(Dense(16, activation='relu'))
two_fortynine_neuron.add(Dense(8, activation='relu'))
two_fortynine_neuron.add(Dense(1, activation='sigmoid'))
print(two_fortynine_neuron.summary())

#Check for cycles in the Sequential Model
two_fortynine_neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'two_fortynine_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_249 = two_fortynine_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 128)               3456      
                                                                 
 dense_43 (Dense)            (None, 64)                8256      
                                                                 
 dense_44 (Dense)            (None, 32)                2080      
                                                                 
 dense_45 (Dense)            (None, 16)                528       
                                                                 
 dense_46 (Dense)            (None, 8)                 136       
                                                                 
 dense_47 (Dense)            (None, 1)                 9         
                                                                 
Total params: 14465 (56.50 KB)
Trainable params: 1446

In [78]:
history_249 = two_fortynine_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
834/852 [============================>.] - ETA: 0s - loss: 0.2999 - accuracy: 0.8508
Epoch 1: val_loss did not improve from 0.27824
852/852 [==============================] - 2s 2ms/step - loss: 0.3004 - accuracy: 0.8502 - val_loss: 0.2989 - val_accuracy: 0.8570
Epoch 2/150
836/852 [============================>.] - ETA: 0s - loss: 0.3136 - accuracy: 0.8513
Epoch 2: val_loss did not improve from 0.27824
852/852 [==============================] - 2s 2ms/step - loss: 0.3141 - accuracy: 0.8506 - val_loss: 0.2823 - val_accuracy: 0.8672
Epoch 3/150
833/852 [============================>.] - ETA: 0s - loss: 0.2951 - accuracy: 0.8556
Epoch 3: val_loss did not improve from 0.27824
852/852 [==============================] - 2s 2ms/step - loss: 0.2955 - accuracy: 0.8552 - val_loss: 0.2939 - val_accuracy: 0.8571
Epoch 4/150
844/852 [============================>.] - ETA: 0s - loss: 0.2938 - accuracy: 0.8515
Epoch 4: val_loss did not improve from 0.27824
852/852 [======================

In [80]:
history_249 = two_fortynine_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
829/852 [============================>.] - ETA: 0s - loss: 0.1963 - accuracy: 0.9058
Epoch 1: val_loss improved from 0.23781 to 0.17656, saving model to four_ninetyseven_model.hdf5
852/852 [==============================] - 2s 2ms/step - loss: 0.1962 - accuracy: 0.9062 - val_loss: 0.1766 - val_accuracy: 0.9165
Epoch 2/150
813/852 [===========================>..] - ETA: 0s - loss: 0.2112 - accuracy: 0.8995
Epoch 2: val_loss did not improve from 0.17656
852/852 [==============================] - 2s 2ms/step - loss: 0.2106 - accuracy: 0.9002 - val_loss: 0.1948 - val_accuracy: 0.9068
Epoch 3/150
821/852 [===========================>..] - ETA: 0s - loss: 0.2011 - accuracy: 0.9041
Epoch 3: val_loss did not improve from 0.17656
852/852 [==============================] - 2s 2ms/step - loss: 0.2030 - accuracy: 0.9027 - val_loss: 0.2155 - val_accuracy: 0.9003
Epoch 4/150
823/852 [===========================>..] - ETA: 0s - loss: 0.2139 - accuracy: 0.9013
Epoch 4: val_loss did not imp

In [91]:
history_249 = two_fortynine_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
843/852 [============================>.] - ETA: 0s - loss: 0.2184 - accuracy: 0.8975
Epoch 1: val_loss did not improve from 0.14642
852/852 [==============================] - 2s 2ms/step - loss: 0.2190 - accuracy: 0.8973 - val_loss: 0.2181 - val_accuracy: 0.8976
Epoch 2/150
824/852 [============================>.] - ETA: 0s - loss: 0.2267 - accuracy: 0.8945
Epoch 2: val_loss did not improve from 0.14642
852/852 [==============================] - 2s 2ms/step - loss: 0.2256 - accuracy: 0.8949 - val_loss: 0.1876 - val_accuracy: 0.9124
Epoch 3/150
817/852 [===========================>..] - ETA: 0s - loss: 0.2122 - accuracy: 0.9010
Epoch 3: val_loss did not improve from 0.14642
852/852 [==============================] - 2s 2ms/step - loss: 0.2121 - accuracy: 0.9007 - val_loss: 0.1957 - val_accuracy: 0.9074
Epoch 4/150
850/852 [============================>.] - ETA: 0s - loss: 0.2243 - accuracy: 0.8964
Epoch 4: val_loss did not improve from 0.14642
852/852 [======================

In [95]:
two_fortynine_neuron.load_weights('two_fortynine_model.hdf5')

In [96]:
# Other Metrics
two_fortynine_neuron_preds = two_fortynine_neuron.predict(X)
accuracy = accuracy_score(y, two_fortynine_neuron_preds.round())
precision = precision_score(y, two_fortynine_neuron_preds.round())
recall = recall_score(y, two_fortynine_neuron_preds.round())
f1score = f1_score(y, two_fortynine_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 978us/step
Accuracy: 0.91
Precision: 0.92
Recall: 0.91
F1-score: 0.91


#### 2.9 Build a Model with 497 Neurons in 6 Layers

In [81]:
# constructing a 497 neuron model
four_ninetyseven_neuron = Sequential()
four_ninetyseven_neuron.add(Dense(256, input_dim = len(X.columns), activation='relu'))
four_ninetyseven_neuron.add(Dense(128, activation='relu'))
four_ninetyseven_neuron.add(Dense(64, activation='relu'))
four_ninetyseven_neuron.add(Dense(32, activation='relu'))
four_ninetyseven_neuron.add(Dense(16, activation='relu'))
four_ninetyseven_neuron.add(Dense(1, activation='sigmoid'))
print(four_ninetyseven_neuron.summary())

#Check for cycles in the Sequential Model
four_ninetyseven_neuron.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callback_a = ModelCheckpoint(filepath = 'four_ninetyseven_model.hdf5', monitor='val_loss', save_best_only = True, save_weights_only = True, verbose = 1)
callback_b = EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose=1)
history_497 = four_ninetyseven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 256)               6912      
                                                                 
 dense_55 (Dense)            (None, 128)               32896     
                                                                 
 dense_56 (Dense)            (None, 64)                8256      
                                                                 
 dense_57 (Dense)            (None, 32)                2080      
                                                                 
 dense_58 (Dense)            (None, 16)                528       
                                                                 
 dense_59 (Dense)            (None, 1)                 17        
                                                                 
Total params: 50689 (198.00 KB)
Trainable params: 506

In [82]:
history_497 = four_ninetyseven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
845/852 [============================>.] - ETA: 0s - loss: 0.2693 - accuracy: 0.8691
Epoch 1: val_loss improved from 0.24381 to 0.23592, saving model to four_ninetyseven_model.hdf5
852/852 [==============================] - 2s 2ms/step - loss: 0.2691 - accuracy: 0.8692 - val_loss: 0.2359 - val_accuracy: 0.8857
Epoch 2/150
851/852 [============================>.] - ETA: 0s - loss: 0.2683 - accuracy: 0.8682
Epoch 2: val_loss did not improve from 0.23592
852/852 [==============================] - 2s 3ms/step - loss: 0.2683 - accuracy: 0.8681 - val_loss: 0.2446 - val_accuracy: 0.8801
Epoch 3/150
831/852 [============================>.] - ETA: 0s - loss: 0.2582 - accuracy: 0.8718
Epoch 3: val_loss did not improve from 0.23592
852/852 [==============================] - 2s 2ms/step - loss: 0.2583 - accuracy: 0.8724 - val_loss: 0.2375 - val_accuracy: 0.8823
Epoch 4/150
831/852 [============================>.] - ETA: 0s - loss: 0.2742 - accuracy: 0.8684
Epoch 4: val_loss did not imp

In [92]:
history_497 = four_ninetyseven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a, callback_b])

Epoch 1/150
835/852 [============================>.] - ETA: 0s - loss: 0.1818 - accuracy: 0.9134
Epoch 1: val_loss did not improve from 0.14642
852/852 [==============================] - 2s 2ms/step - loss: 0.1816 - accuracy: 0.9137 - val_loss: 0.1564 - val_accuracy: 0.9253
Epoch 2/150
814/852 [===========================>..] - ETA: 0s - loss: 0.1690 - accuracy: 0.9201
Epoch 2: val_loss did not improve from 0.14642
852/852 [==============================] - 2s 2ms/step - loss: 0.1707 - accuracy: 0.9182 - val_loss: 0.1505 - val_accuracy: 0.9307
Epoch 3/150
811/852 [===========================>..] - ETA: 0s - loss: 0.1665 - accuracy: 0.9192
Epoch 3: val_loss did not improve from 0.14642
852/852 [==============================] - 2s 2ms/step - loss: 0.1693 - accuracy: 0.9178 - val_loss: 0.1581 - val_accuracy: 0.9268
Epoch 4/150
849/852 [============================>.] - ETA: 0s - loss: 0.1730 - accuracy: 0.9144
Epoch 4: val_loss did not improve from 0.14642
852/852 [======================

In [97]:
history_497 = four_ninetyseven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a])

Epoch 1/150
810/852 [===========================>..] - ETA: 0s - loss: 0.1720 - accuracy: 0.9180
Epoch 1: val_loss did not improve from 0.13964
852/852 [==============================] - 2s 2ms/step - loss: 0.1737 - accuracy: 0.9169 - val_loss: 0.1569 - val_accuracy: 0.9267
Epoch 2/150
852/852 [==============================] - ETA: 0s - loss: 0.1735 - accuracy: 0.9183
Epoch 2: val_loss did not improve from 0.13964
852/852 [==============================] - 2s 2ms/step - loss: 0.1735 - accuracy: 0.9183 - val_loss: 0.1451 - val_accuracy: 0.9326
Epoch 3/150
840/852 [============================>.] - ETA: 0s - loss: 0.1687 - accuracy: 0.9232
Epoch 3: val_loss did not improve from 0.13964
852/852 [==============================] - 2s 2ms/step - loss: 0.1692 - accuracy: 0.9229 - val_loss: 0.1802 - val_accuracy: 0.9185
Epoch 4/150
825/852 [============================>.] - ETA: 0s - loss: 0.1799 - accuracy: 0.9178
Epoch 4: val_loss did not improve from 0.13964
852/852 [======================

In [98]:
four_ninetyseven_neuron.load_weights('four_ninetyseven_model.hdf5')

In [99]:
# Other Metrics
four_ninetyseven_neuron_preds = four_ninetyseven_neuron.predict(X)
accuracy = accuracy_score(y, four_ninetyseven_neuron_preds.round())
precision = precision_score(y, four_ninetyseven_neuron_preds.round())
recall = recall_score(y,four_ninetyseven_neuron_preds.round())
f1score = f1_score(y, four_ninetyseven_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 938us/step
Accuracy: 0.95
Precision: 0.96
Recall: 0.94
F1-score: 0.95


In [100]:
history_497 = four_ninetyseven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a])

Epoch 1/150
842/852 [============================>.] - ETA: 0s - loss: 0.1298 - accuracy: 0.9387
Epoch 1: val_loss did not improve from 0.10524
852/852 [==============================] - 2s 2ms/step - loss: 0.1298 - accuracy: 0.9387 - val_loss: 0.1151 - val_accuracy: 0.9469
Epoch 2/150
826/852 [============================>.] - ETA: 0s - loss: 0.1339 - accuracy: 0.9378
Epoch 2: val_loss did not improve from 0.10524
852/852 [==============================] - 2s 2ms/step - loss: 0.1360 - accuracy: 0.9365 - val_loss: 0.1152 - val_accuracy: 0.9476
Epoch 3/150
829/852 [============================>.] - ETA: 0s - loss: 0.1395 - accuracy: 0.9345
Epoch 3: val_loss did not improve from 0.10524
852/852 [==============================] - 2s 2ms/step - loss: 0.1388 - accuracy: 0.9349 - val_loss: 0.1206 - val_accuracy: 0.9461
Epoch 4/150
827/852 [============================>.] - ETA: 0s - loss: 0.1231 - accuracy: 0.9422
Epoch 4: val_loss did not improve from 0.10524
852/852 [======================

In [103]:
history_497 = four_ninetyseven_neuron.fit(X, y, validation_data=(X, y), epochs=150, batch_size=10, callbacks = [callback_a])

Epoch 1/150
833/852 [============================>.] - ETA: 0s - loss: 0.1035 - accuracy: 0.9501
Epoch 1: val_loss did not improve from 0.09105
852/852 [==============================] - 3s 3ms/step - loss: 0.1032 - accuracy: 0.9499 - val_loss: 0.0986 - val_accuracy: 0.9521
Epoch 2/150
824/852 [============================>.] - ETA: 0s - loss: 0.1142 - accuracy: 0.9447
Epoch 2: val_loss did not improve from 0.09105
852/852 [==============================] - 2s 3ms/step - loss: 0.1143 - accuracy: 0.9446 - val_loss: 0.1273 - val_accuracy: 0.9434
Epoch 3/150
842/852 [============================>.] - ETA: 0s - loss: 0.1212 - accuracy: 0.9418
Epoch 3: val_loss did not improve from 0.09105
852/852 [==============================] - 3s 3ms/step - loss: 0.1216 - accuracy: 0.9415 - val_loss: 0.1117 - val_accuracy: 0.9513
Epoch 4/150
848/852 [============================>.] - ETA: 0s - loss: 0.1234 - accuracy: 0.9433
Epoch 4: val_loss did not improve from 0.09105
852/852 [======================

In [104]:
four_ninetyseven_neuron.load_weights('four_ninetyseven_model.hdf5')

In [105]:
# Other Metrics
four_ninetyseven_neuron_preds = four_ninetyseven_neuron.predict(X)
accuracy = accuracy_score(y, four_ninetyseven_neuron_preds.round())
precision = precision_score(y, four_ninetyseven_neuron_preds.round())
recall = recall_score(y,four_ninetyseven_neuron_preds.round())
f1score = f1_score(y, four_ninetyseven_neuron_preds.round())
print("Accuracy: %.2f" % (accuracy))
print("Precision: %.2f" % (precision))
print("Recall: %.2f" % (recall))
print("F1-score: %.2f" % (f1score))

267/267 [==============================] - 0s 1ms/step
Accuracy: 0.96
Precision: 0.96
Recall: 0.96
F1-score: 0.96


#### Underfitting vs. Overfitting in a Neural Network
Underfit models have a high bias meaning they produce innaccurate results for the training data and test data sets. While overfit models have a high variance which means that they produce accurate results for the training set but not for the test set.

##### Phase 2 Conclusion
I really struggled with this phase, I spent hours tweaking and removing and trying different input variable combinations trying to improve my accurracy in my initial models and was ultimately unsuccessful. I also attempted to adjust the learning speed to slower speeds such as 0.01,0.001, 0,0001, and 0.00001 as well as adjusting the batch sizes to 10, 20, 50, and number of epochs from 20, 60, 100, and 150 but accuracy for the 1 neuron model remained at the same 22%, the 3 neuron model with 2 layers had about 18% accuracy and the 7 neuron model with 3 layers had such a small accuracy it appeared to be 0 even in scientific notation, with some adjustments I was able to get the 7 neuron model accuracy up to 19%. Ultimately I moved on because it appeared that when using all the data my model was immediately overfit and performed poorly no matter what I attempted. My conclusion at this time was perhaps the data had too much noise so it was already overfit and that was why it had such poor results. <br> 
I eventually moved onto Phase 3 and tried using some different normalization processes and had better results on those models. I eventually returned and redid Phase 2 with the same methods of normalization that worked best in Phase 3 and immediately had better results. There must have been something wrong with my initial normalization in the first models I built so they were not performing properly. With the corrected normalization my one neuron model was able to get up to about 72% accuracy pretty quickly. From there my other 8 larger models all improved, where as my initial poor models were decreasing in accuracy. 
<br>
Due to time constraints, and having returned to improve this phase I am stopping with my last overfit model attempt with its 96% accuracy. Unfortunately I did not make it to 100% accurancy on any of these models but I do now feel more confident that I could identify an overfit model and am now more aware of the factors that can lead to an overfit model. 